# MobileNet

## Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

## MobileNet

### Depthwise Separable Convolution

In [2]:
# 3x3 Depthwise
def depthwise(layer, strides=1, name=0):
    if strides == 2:
        x = keras.layers.ZeroPadding2D(((0,1), (0,1)),
                                        name=f'Depthwise_padding{name}')(layer)
    else:
        x = layer
    x = keras.layers.DepthwiseConv2D(kernel_size=(3,3),
                                     strides=strides,
                                     padding='same' if strides==1 else 'valid',
                                     use_bias=False,
                                     name=f'Depthwise_{name}')(x)
    x = keras.layers.BatchNormalization(name=f'Depthwise_batchnorm_{name}')(x)
    x = keras.layers.ReLU(6., name=f'Depthwise_relu_{name}')(x)
    return x

In [3]:
# 1x1 Pointwise
def pointwise(layer, channel, width_multiplier=1, strides=1, name=0):
    x = keras.layers.Conv2D(filters=int(round(channel*width_multiplier)),
                            kernel_size=(1,1),
                            strides=strides,
                            padding='same',
                            use_bias=False,
                            name=f"Pointwise_{name}")(layer)
    x = keras.layers.BatchNormalization(name=f'Pointwise_batchnorm_{name}')(x)
    x = keras.layers.ReLU(6., name=f'Pointwise_relu_{name}')(x)
    
    return x

### Build MobileNet

In [4]:
def build_MobileNet(input_shape=(224,224,3),
                    num_strides_list=[1, 2, 1, 2, 1, 2, 1, 2, 1],
                    num_channel_list=[64, 128, 128, 256, 256, 512, 512, 1024, 1024],
                    width_multiplier = 1,
                    resolution_multiplier = 1,
                    num_classes=1000):
    input_resolution = (int(round(input_shape[0]*resolution_multiplier)))
    
    input_layer = keras.layers.Input(shape=(input_resolution, input_resolution, 3), name='Input')
    output = keras.layers.ZeroPadding2D(((0,1), (0,1)),
                                        name='First_padding')(input_layer)
    output = keras.layers.Conv2D(filters=32,
                                 kernel_size=(3,3),
                                 strides=2,
                                 padding='valid',
                                 use_bias=False,
                                 name='First_Conv_layer')(output)
    output = keras.layers.BatchNormalization(name='First_BatchNorm')(output)
    output = keras.layers.ReLU(6., name='First_Activation')(output)
    
    # Depthwise Separable Convolution
    for idx, (stride, channel) in enumerate(zip(num_strides_list, num_channel_list)):
        if idx == 6:
            for j in range(5):
                output = depthwise(output, strides= stride, name=f'{idx}_{j}')
                output = pointwise(output, channel, width_multiplier, name=f'{idx}_{j}')
        else:
            output = depthwise(output, strides= stride, name=idx)
            output = pointwise(output, channel, width_multiplier, name=idx)
    
    
    output = keras.layers.GlobalAveragePooling2D(name='GAP')(output)
    output = keras.layers.Dropout(0.001, name='Dropout')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='Classifier')(output)
    
    model = keras.Model(inputs=input_layer, outputs=output)
    
    return model

In [5]:
MobileNet = build_MobileNet()
MobileNet.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
First_padding (ZeroPadding2D (None, 225, 225, 3)       0         
_________________________________________________________________
First_Conv_layer (Conv2D)    (None, 112, 112, 32)      864       
_________________________________________________________________
First_BatchNorm (BatchNormal (None, 112, 112, 32)      128       
_________________________________________________________________
First_Activation (ReLU)      (None, 112, 112, 32)      0         
_________________________________________________________________
Depthwise_0 (DepthwiseConv2D (None, 112, 112, 32)      288       
_________________________________________________________________
Depthwise_batchnorm_0 (Batch (None, 112, 112, 32)     